Table of Contents

1  1. Numpy Array to dataframe
1.1  1-1. Original Data Load
1.2  1-2. Array to DF
1.3  1-3 DF to Influx
1.3.1  Column Modify
1.3.2  Data Local Save : Classification
1.3.3  Original Data Upload
1.3.3.1  이 위에까지 돌려보시면 됩니다~
1.3.4  Data Split
2  2. dataFrame to Numpy Array
2.1  2-1. Influx Data Preparation
2.2  2-2. DF to Numpy Array

In [ ]:
import sys

sys.path.append("../../..")
sys.path.append("../../../../")

1. Numpy Array to dataframe

1-1. Original Data Load

In [ ]:
problem = "classification" # classification  #regression

from Clust.clust.transformation.type.NPArrayToDF import trans3NPtoDF
import pickle
datafolderAddress = './ku_data/{}Test/'.format(problem)
train_x = pickle.load(open(datafolderAddress+'x_train.pkl', 'rb'))
train_y = pickle.load(open(datafolderAddress+'y_train.pkl', 'rb'))
test_x = pickle.load(open(datafolderAddress+'x_test.pkl', 'rb'))
test_y = pickle.load(open(datafolderAddress+'y_test.pkl', 'rb'))

In [ ]:
train_x.shape

In [ ]:
train_y.shape

1-2. Array to DF

In [ ]:
if problem == "regression":
    trainDF_X, trainDF_y = trans3NPtoDF(train_x, train_y, '2016-1-11')
    testDF_X, testDF_y = trans3NPtoDF(test_x, test_y, '2021-1-1')
elif problem == "classification":
    trainDF_X, trainDF_y = trans3NPtoDF(train_x, train_y, '1992-1-11')
    testDF_X, testDF_y = trans3NPtoDF(test_x, test_y, '2013-1-1')

In [ ]:
trainDF_X.shape

1-3 DF to Influx

Column Modify

In [ ]:
if problem == "regression":
    # columnName 수정
    rename_columns = {"col_0":"T1", "col_1":"RH_1", "col_2":"T2", "col_3":"RH_2", "col_4":"T3", "col_5":"RH_3", "col_6":"T4", "col_7":"RH_4",
    "col_8":"T5", "col_9":"RH_5", "col_10":"T6", "col_11":"RH_6", "col_12":"T7", "col_13":"RH_7", "col_14":"T8", "col_15":"RH_8",
    "col_16":"T9", "col_17":"RH_9", "col_18":"T_out", "col_19":"Press_mm_hg", "col_20":"RH_out", "col_21":"Windspeed", "col_22":"Visibility",
    "col_23":"Tdewpoint"}

    trainDF_X.rename(columns = rename_columns, inplace = True)
    testDF_X.rename(columns = rename_columns, inplace = True)

In [ ]:
import pandas as pd
trainDF_y['value'] = pd.to_numeric(trainDF_y['value'],errors = 'coerce')
testDF_y['value'] = pd.to_numeric(testDF_y['value'],errors = 'coerce')

In [ ]:
trainDF_X.head(2)

In [ ]:
trainDF_X.info()

In [ ]:
trainDF_y.info()

In [ ]:
testDF_X.info()

In [ ]:
testDF_y.info()

Data Local Save : Classification

In [ ]:
if problem =="classification":
    trainDF_X.to_csv("classification_trainDF_X.csv")
    testDF_X.to_csv("classification_testDF_X.csv")
    trainDS_X = pd.read_csv("classification_trainDF_X.csv", chunksize=25000, index_col = "Unnamed: 0")
    testDS_X = pd.read_csv("classification_testDF_X.csv", chunksize=25000, index_col = "Unnamed: 0")

In [ ]:
trainDS_X

In [ ]:
testDS_X

Original Data Upload

In [ ]:
from Clust.setting.influx_setting_KETI import CLUSTDataServer2 as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC

db_client = iC.InfluxClient(ins)

In [ ]:
# 1. Original Upload
if problem =="regression":
     # Train X
    DBName = "life_indoor_environment"
    MSName = "applianceEnergyDatasetTrainX_10min"
    db_client.write_db(DBName, MSName, trainDF_X)
    
    # Train y
    DBName = "life_indoor_environment"
    MSName = "applianceEnergyDatasetTrainy_1day"
    db_client.write_db(DBName, MSName, trainDF_y)
    
    # Test X
    DBName = "life_indoor_environment"
    MSName = "applianceEnergyDatasetTestX_10min"
    db_client.write_db(DBName, MSName, testDF_X)
    
    # Test y
    DBName = "life_indoor_environment"
    MSName = "applianceEnergyDatasetTesty_1day"
    db_client.write_db(DBName, MSName, testDF_y)

elif problem =="classification":
    print("Start Classification Data Save")
    
    print("Start Data 1")
    # Test y 
    DBName = "life_action_pattern"
    MSName = "patternNameTest"
    db_client.write_db(DBName, MSName, testDF_y)
    
    print("Start Data 2")
    # Train y 
    DBName = "life_action_pattern"
    MSName = "patternNameTrain"
    db_client.write_db(DBName, MSName, trainDF_y)
    
    import time
    time.sleep(180)

In [ ]:
if problem =="classification":
    print("Start Data 3")
    # Train X
    for train_X in trainDS_X:
        DBName = "bio_action_sensors"
        MSName = "accelerationTrain"
        db_client.write_db(DBName, MSName, train_X)
    
    print("Start Data 4")
    # Test X
    for test_X in testDS_X:
        DBName = "bio_action_sensors"
        MSName = "accelerationTest"
        db_client.write_db(DBName, MSName, test_X)

Data Split

In [ ]:
if problem =="regression":
    trainDF_X_Temp = trainDF_X[["T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9"]]
    trainDF_X_Humi = trainDF_X[["RH_1", "RH_2", "RH_3", "RH_4", "RH_5", "RH_6", "RH_7", "RH_8", "RH_9"]]
    trainDF_X_Out = trainDF_X[["T_out", "Press_mm_hg" ,"RH_out" ,"Windspeed" ,"Visibility" ,"Tdewpoint"]]

    testDF_X_Temp = testDF_X[["T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9"]]
    testDF_X_Humi = testDF_X[["RH_1", "RH_2", "RH_3", "RH_4", "RH_5", "RH_6", "RH_7", "RH_8", "RH_9"]]
    testDF_X_Out = testDF_X[["T_out", "Press_mm_hg" ,"RH_out" ,"Windspeed" ,"Visibility" ,"Tdewpoint"]]

In [ ]:
# 2. Split Data Upload

if problem =="regression":
    # Train X
    DBName = "life_indoor_environment"
    MSName = "humidityTrain_10min"
    db_client.write_db(DBName, MSName, trainDF_X_Humi)
    
    DBName = "life_indoor_environment"
    MSName = "temperatureTrain_10min"
    db_client.write_db(DBName, MSName, trainDF_X_Temp)
    
    DBName = "weather_outdoor_environment"
    MSName = "belgiumChieverseAirportTrain_10min"
    db_client.write_db(DBName, MSName, trainDF_X_Out)
    
    # Test X
    DBName = "life_indoor_environment"
    MSName = "humidityTest_10min"
    db_client.write_db(DBName, MSName, testDF_X_Humi)
    
    DBName = "life_indoor_environment"
    MSName = "temperatureTest_10min"
    db_client.write_db(DBName, MSName, testDF_X_Temp)
    
    DBName = "weather_outdoor_environment"
    MSName = "belgiumChieverseAirportTest_10min"
    db_client.write_db(DBName, MSName, testDF_X_Out)

2. dataFrame to Numpy Array

2-1. Influx Data Preparation

In [ ]:
# regression Original Data
# Train X
DBName = "life_indoor_environment"
MSName = "applianceEnergyDatasetTrainX_10min"
trainDF_X = db_client.get_data(DBName, MSName)

# Train y
DBName = "life_indoor_environment"
MSName = "applianceEnergyDatasetTrainy_1day"
trainDF_y = db_client.get_data(DBName, MSName)

In [ ]:
trainDF_X.shape

2-2. DF to Numpy Array

In [ ]:
from Clust.clust.transformation.type.DFToNPArray import transDFtoNP
dfX = trainDF_X 
dfy = trainDF_y
X, y = transDFtoNP(dfX, dfy)

In [ ]:
X.shape

In [ ]:
y.shape